In [13]:
import itertools

from sklearn.datasets import load_breast_cancer, load_iris
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import autograd.numpy as np
from autograd import grad

from GradientDescent import Stochastic
import utils

In [14]:
# def create_layers(network_input_size, layer_output_sizes):
#     layers = []

#     i_size = network_input_size
#     for layer_output_size in layer_output_sizes:
#         W = np.random.randn(layer_output_size, i_size).T * 0.001
#         b = np.random.randn(layer_output_size)
#         layers.append((W, b))

#         i_size = layer_output_size
#     return layers

In [15]:
# def feed_forward(inputs, layers, activation_funcs):
#     a = inputs
#     for (W, b), activation_func in zip(layers, activation_funcs):
#         z = a @ W + b
#         a = activation_func(z)
#     return a

In [16]:
# def sigmoid(z):
#     neg = np.where(z < 0, np.exp(z) / (1 + np.exp(z)), 0)
#     pos = np.where(z >= 0, 1 / (1 + np.exp(-z)), 0)
#     return pos + neg

In [17]:
# def cross_entropy(predict, target):
#     predict = np.clip(predict, 1e-8, 1 - 1e-8)
#     return np.sum(-target * np.log(predict))

# def cost(input, layers, activation_funcs, target):
#     prediction = feed_forward(input, layers, activation_funcs)
#     return cross_entropy(prediction, target)

In [18]:
# network_input_size = 30
# layer_output_sizes = [1]
# activation_funcs = [sigmoid]
# backprop = grad(cost, 1)

In [19]:
# cancer = load_breast_cancer()
# inputs = cancer.data
# targets = cancer.target


# x_train, x_test, y_train, y_test = train_test_split(inputs, targets)
# x_train, x_test, y_train, y_test = x_train[:,:], x_test[:,:], y_train[:], y_test[:]

# scaler = StandardScaler()

# x_train = scaler.fit_transform(x_train, y_train)
# x_test = scaler.transform(x_test)

# layers = create_layers(network_input_size, layer_output_sizes)

In [20]:
# lr = 0.001
# for i in range(10000):
#     gradient = backprop(x_train, layers, activation_funcs, y_train)
#     dW, db = gradient[0]
#     W, b = layers[0]
#     W -= lr * dW
#     b -= lr * db
#     layers = [(W, b)]

#     if int(i % 10) == 0:
#         prediction = feed_forward(x_train, layers, activation_funcs)
#         print(cross_entropy(prediction, y_train))

# prediction = feed_forward(x_train, layers, activation_funcs)

In [21]:
# cold = np.where(prediction >= 0.5, 1, 0) + np.where(prediction < 0.5, 1, 0)

In [22]:
# for elem, targ in zip(cold, y_train):
#     print(f"pred: {elem[0]}       targ: {targ}")

# for pred, targ in zip(prediction, y_train):
#     print(f"pred: {pred[0]}       targ: {targ}")

In [26]:
def create_layers(network_input_size, layer_output_sizes):
    layers = []
    i_size = network_input_size
    for layer_output_size in layer_output_sizes:
        std = np.sqrt(2 / (layer_output_size + i_size))
        W = np.random.normal(scale=std, size=(i_size, layer_output_size))
        b = np.zeros(layer_output_size)
        layers.append((W, b))
    return layers

def feed_forward(inputs, layers, activation_funcs):
    a = inputs
    for (W, b), activation_func in zip(layers, activation_funcs):
        z = a @ W + b
        a = activation_func(z)
    return a

def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def cross_entropy(predict, target):
    predict = np.clip(predict, 1e-10, 1 - 1e-10)
    target = target
    return -np.sum(target * np.log(predict) + (1 - target) * np.log(1 - predict))

network_input_size = 30
n_layers = 5
layer_output_sizes = [100] * (n_layers-1) + [1]
activation_funcs = [sigmoid] * n_layers

def cost(input, layers, target):
    prediction = feed_forward(input, layers, activation_funcs)
    return cross_entropy(prediction, target)

inputs, targets = utils.get_cancer_data()

x_train, x_test, y_train, y_test = train_test_split(inputs, targets)
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

layers = create_layers(network_input_size, layer_output_sizes)

lr = 0.0001

backprop = grad(cost, 1)

for i in range(10000):
    gradient = backprop(x_train, layers, y_train)

    # Update weights and biases for all layers
    for j in range(len(layers)):
        dW, db = gradient[j]
        
        # Clip gradients
        dW = np.clip(dW, -1, 1)
        db = np.clip(db, -1, 1)
        
        W, b = layers[j]
        W -= lr * dW
        b -= lr * db
        layers[j] = (W, b)

    if i % 1000 == 0:
        prediction = feed_forward(x_train, layers, activation_funcs)
        print(f"Iteration {i}, Cost: {cross_entropy(prediction, y_train)}")


prediction = feed_forward(x_train, layers, activation_funcs)

for pred, targ in zip(prediction, y_train):
    print(f"prediction: {pred}       target: {targ}")

print(f"Final Cost: {cross_entropy(prediction, y_train)}")


ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 30 is different from 100)